In [1]:
# ===== 셀 1: 환경 로딩 =====
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# ===== 셀 2: 샘플 문서 준비 =====
# 실제로는 PDF 로딩하지만, 테스트용 샘플 텍스트
sample_docs = [
    """
    [NH투자증권 리포트 - 2025.10.15]
    삼성전자 목표주가: 90,000원
    투자의견: 매수
    AI 반도체 시장에서 삼성전자는 NVIDIA 대비 경쟁력을 확보하고 있습니다.
    HBM3E 양산이 시작되면서 수익성 개선이 예상됩니다.
    """,
    """
    [한국투자증권 리포트 - 2025.10.10]
    네이버 목표주가: 250,000원
    투자의견: 매수
    AI 검색 서비스 확대로 광고 매출 증가가 기대됩니다.
    """,
    """
    [미래에셋증권 리포트 - 2025.10.12]
    현대차 목표주가: 280,000원
    투자의견: 중립
    전기차 시장 경쟁 심화로 단기 실적 압박이 예상됩니다.
    """
]

# ===== 셀 3: 텍스트 청킹 =====
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# Document 객체로 변환
documents = [Document(page_content=text) for text in sample_docs]

# 청킹 (실제로는 큰 PDF라서 필요, 여기선 작아서 의미 없음)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(documents)
print(f"총 {len(chunks)}개 청크 생성")

# ===== 셀 4: 임베딩 및 ChromaDB 저장 =====
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# 임베딩 생성
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# ChromaDB 생성 (테스트용 별도 디렉토리)
db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./embeddings/test_db",
    collection_name="test_reports"
)
print("ChromaDB 생성 완료!")

# ===== 셀 5: 유사도 검색 테스트 =====
query = "삼성전자 AI 반도체 전망은?"
results = db.similarity_search(query, k=2)

print(f"질문: {query}\n")
for i, doc in enumerate(results, 1):
    print(f"[검색 결과 {i}]")
    print(doc.page_content)
    print()

# ===== 셀 6: RAG 체인 구축 =====
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# LLM
llm = ChatOpenAI(
    model="gpt-4-turbo-preview",
    temperature=0.3,
    openai_api_key=api_key
)

# Retriever
retriever = db.as_retriever(search_kwargs={"k": 2})

# 프롬프트
prompt_template = """
당신은 전문 투자 상담가입니다.
아래 증권사 리포트를 참고하여 질문에 답변하세요.

참고 문서:
{context}

질문: {question}

답변 지침:
1. 참고 문서의 내용을 기반으로 정확히 답변하세요
2. 출처를 명확히 밝히세요 (예: "NH투자증권 리포트에 따르면...")
3. 초보 투자자도 이해할 수 있도록 쉽게 설명하세요

답변:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

# RAG Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

# ===== 셀 7: RAG 질의 테스트 =====
test_questions = [
    "삼성전자의 AI 반도체 전망은?",
    "네이버 주가 목표가는?",
    "현대차 투자의견은?"
]

for q in test_questions:
    print(f"\n{'='*60}")
    print(f"질문: {q}")
    print('='*60)
    
    result = qa_chain({"query": q})
    answer = result["result"]
    sources = result["source_documents"]
    
    print(f"\n답변:\n{answer}\n")
    
    print("출처:")
    for i, doc in enumerate(sources, 1):
        print(f"  [{i}] {doc.page_content[:100]}...")


총 3개 청크 생성


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


ChromaDB 생성 완료!


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


질문: 삼성전자 AI 반도체 전망은?

[검색 결과 1]
[NH투자증권 리포트 - 2025.10.15]
    삼성전자 목표주가: 90,000원
    투자의견: 매수
    AI 반도체 시장에서 삼성전자는 NVIDIA 대비 경쟁력을 확보하고 있습니다.
    HBM3E 양산이 시작되면서 수익성 개선이 예상됩니다.

[검색 결과 2]
[NH투자증권 리포트 - 2025.10.15]
    삼성전자 목표주가: 90,000원
    투자의견: 매수
    AI 반도체 시장에서 삼성전자는 NVIDIA 대비 경쟁력을 확보하고 있습니다.
    HBM3E 양산이 시작되면서 수익성 개선이 예상됩니다.


질문: 삼성전자의 AI 반도체 전망은?


C:\Users\user\AppData\Local\Temp\ipykernel_25172\2214989669.py:132: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": q})



답변:
삼성전자의 AI 반도체 전망에 대해 NH투자증권 리포트(2025.10.15)에 따르면, 삼성전자는 AI 반도체 시장에서 NVIDIA와의 비교에서도 경쟁력을 확보하고 있는 것으로 평가받고 있습니다. 이는 삼성전자가 AI 반도체 분야에서 중요한 플레이어로 자리매김하고 있음을 의미합니다. 또한, 리포트는 삼성전자가 HBM3E를 양산하기 시작하면서 수익성 개선이 예상된다고 언급하고 있습니다. HBM3E는 고성능 컴퓨팅과 AI 분야에서 요구되는 고대역폭 메모리 기술로, 이의 양산은 삼성전자가 해당 시장에서 더욱 강력한 위치를 확보할 수 있음을 시사합니다. 따라서, 삼성전자의 AI 반도체 사업은 긍정적인 전망을 가지고 있으며, 이는 투자자에게 매력적인 기회가 될 수 있습니다.

출처:
  [1] [NH투자증권 리포트 - 2025.10.15]
    삼성전자 목표주가: 90,000원
    투자의견: 매수
    AI 반도체 시장에서 삼성전자는 NVIDIA 대비 경쟁력을 확...
  [2] [NH투자증권 리포트 - 2025.10.15]
    삼성전자 목표주가: 90,000원
    투자의견: 매수
    AI 반도체 시장에서 삼성전자는 NVIDIA 대비 경쟁력을 확...

질문: 네이버 주가 목표가는?

답변:
네이버의 주가 목표가는 250,000원입니다. 이 정보는 한국투자증권 리포트에 기반한 것으로, 네이버가 AI 검색 서비스를 확대함에 따라 광고 매출의 증가가 기대된다고 평가되었습니다. 간단히 말해, 네이버가 더 스마트한 검색 기능을 제공하여 더 많은 사용자를 끌어들이고, 이로 인해 광고로 더 많은 수익을 창출할 것으로 보인다는 의미입니다.

출처:
  [1] [한국투자증권 리포트 - 2025.10.10]
    네이버 목표주가: 250,000원
    투자의견: 매수
    AI 검색 서비스 확대로 광고 매출 증가가 기대됩니다....
  [2] [한국투자증권 리포트 - 2025.10.10]
    네이버 목표주가: 250,000원
    투자의견: 매수
